# Keras IMDb (Using MLP, RNN, LSTM) 

尚缺 U14 待完成

## <font color='dark'>Import library</font>

In [4]:
import urllib.request
import os               # 確認檔案存在與否
import tarfile          # 用於壓縮檔案

from keras.preprocessing import sequence       # 用於控制字典的大小，例如字典中字的數量為固定 100 字
from keras.preprocessing.text import Tokenizer # 將內容建成字典

## <font color='dark'>Load Data</font>

### Download data

In [5]:
# Download the IMDN dataset

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath = "data/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url,filepath)   # 檔案不存在，存放於 data 中
    print('Download : ',result)

In [6]:
# 解壓縮檔案，下載檔案

if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("data/aclImdb_v1.tar.gz", 'r:gz')   # 開啟壓縮檔
    result = tfile.extractall('data/')                       # 解壓縮到 data 中

### Read the data

In [7]:
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)

In [8]:
import os

def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []
    
    positive_path = path+filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path+f]
        
    negative_path = path+filetype+"/pos/"
    for f in os.listdir(negative_path):
        file_list += [positive_path+f]
        
    print('read ',filetype, "files : ", len(file_list))
    
    all_labels = ([1]*12500+[0]*12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    return all_labels,all_texts

In [9]:
y_train, train_text = read_files("train")

read  train files :  25000


In [10]:
y_test, test_text = read_files("test")

read  test files :  25000


In [21]:
len(test_text)

25000

### Build the words dict (token)

In [22]:
token = Tokenizer(num_words=200)   # 字典大小為 200 字
token.fit_on_texts(train_text)     # 將我們的文檔餵進我們的字典，保留數量最多的前 200 字，若沒在裡面就當作沒看到

In [25]:
print(token.document_count)

25000


In [31]:
token.word_index   # 這是以出現次數所做的排序

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'as': 12,
 'with': 13,
 'for': 14,
 'was': 15,
 'but': 16,
 'film': 17,
 'movie': 18,
 'his': 19,
 'on': 20,
 'are': 21,
 'he': 22,
 'you': 23,
 'not': 24,
 'one': 25,
 'have': 26,
 'be': 27,
 'by': 28,
 'all': 29,
 'an': 30,
 'at': 31,
 'who': 32,
 'from': 33,
 'her': 34,
 'they': 35,
 'has': 36,
 'so': 37,
 'like': 38,
 "it's": 39,
 'about': 40,
 'very': 41,
 'out': 42,
 'or': 43,
 'good': 44,
 'more': 45,
 'when': 46,
 'some': 47,
 'what': 48,
 'if': 49,
 'just': 50,
 'she': 51,
 'there': 52,
 'story': 53,
 'time': 54,
 'my': 55,
 'great': 56,
 'well': 57,
 'up': 58,
 'which': 59,
 'their': 60,
 'see': 61,
 'can': 62,
 'also': 63,
 'really': 64,
 'would': 65,
 'will': 66,
 'me': 67,
 'had': 68,
 'only': 69,
 'we': 70,
 'him': 71,
 'even': 72,
 'most': 73,
 'other': 74,
 'were': 75,
 'first': 76,
 'than': 77,
 'much': 78,
 'no': 79,
 'its': 80,
 'into': 81,
 'people': 

### words list turn to the number list (according to dictionary)

In [32]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [34]:
print(train_text[0])

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


In [35]:
print(x_train_seq[0])  # is:6  ;  a:3   。  因為有些字太少見，所以自動把它忽略掉

[6, 3, 190, 8, 31, 1, 168, 54, 12, 47, 74, 40, 88, 127, 12, 55, 122, 7, 1, 67, 5, 11, 6, 78, 5, 77, 6, 1, 5, 1, 32, 62, 61, 197, 141, 60, 1, 4, 1, 29, 67, 4, 1, 9, 2, 60, 46, 9, 195, 1, 7, 59, 3, 5, 192, 1, 9, 31, 3, 180, 126, 5, 25, 4, 135, 5, 9, 11, 96, 4, 55, 98, 11, 6, 48, 3, 11, 8]


### modify the length of the number list

In [36]:
# 有些字數會超過我們預期的 imput ，有些則是會少於，這時我們需要做一些調整
# 少的在頭幾個補 0，多的直接去掉頭幾個

x_train = sequence.pad_sequences(x_train_seq,maxlen=100)
x_test_seq = sequence.pad_sequences(x_test_seq,maxlen=100)